# PyTorch Neural Networks Demo - NYC Parking Fines

This notebook demonstrates how to use the PyTorch neural network models for parking fines prediction.

## Prerequisites

Before running this notebook:
1. Install PyTorch dependencies: `pip install -r requirements_pytorch.txt`
2. Train the models: `python train_pytorch_model.py`

This will create trained models in the `models/` directory.

In [ ]:
# Import required libraries
import torch
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pytorch_model import FineAmountRegressor, FineCategoryClassifier

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Load Trained Models

In [ ]:
# Load preprocessor
with open('models/preprocessor.pkl', 'rb') as f:
    preprocessor = pickle.load(f)
print("✓ Preprocessor loaded")

# Load regression model
reg_checkpoint = torch.load('models/pytorch_regressor.pth', map_location='cpu')
reg_model = FineAmountRegressor(input_dim=reg_checkpoint['input_dim'])
reg_model.load_state_dict(reg_checkpoint['model_state_dict'])
reg_model.eval()
print("\n✓ Regression model loaded")
print(f"  MAE: {reg_checkpoint['metrics']['mae']:.2f}")
print(f"  RMSE: {reg_checkpoint['metrics']['rmse']:.2f}")
print(f"  R²: {reg_checkpoint['metrics']['r2']:.4f}")

# Load classification model
class_checkpoint = torch.load('models/pytorch_classifier.pth', map_location='cpu')
class_model = FineCategoryClassifier(
    input_dim=class_checkpoint['input_dim'],
    num_classes=class_checkpoint['num_classes']
)
class_model.load_state_dict(class_checkpoint['model_state_dict'])
class_model.eval()
label_encoder = class_checkpoint['label_encoder']
print("\n✓ Classification model loaded")
print(f"  Accuracy: {class_checkpoint['metrics']['accuracy']:.4f}")
print(f"  F1 Score (macro): {class_checkpoint['metrics']['f1_macro']:.4f}")

## Create Sample Data

Let's create some example parking violations to predict.

In [ ]:
# Sample parking violations
sample_data = pd.DataFrame({
    'Registration State': ['NY', 'NJ', 'NY', 'PA', 'CT'],
    'Plate Type': ['PAS', 'PAS', 'COM', 'PAS', 'PAS'],
    'Vehicle Body Type': ['SUBN', 'SDN', 'VAN', 'SDN', 'SUBN'],
    'Vehicle Make': ['TOYOTA', 'HONDA', 'FORD', 'CHEVR', 'BMW'],
    'Violation County': ['NY', 'NY', 'BX', 'K', 'NY'],
    'Violation In Front Of Or Opposite': ['F', 'O', 'F', 'F', 'O'],
    'Plate ID': ['ABC1234', 'XYZ5678', 'COM999', 'TEST123', 'BMW456'],
    'Vehicle Expiration_year': [2015, 2016, 2015, 2015, 2016],
    'Vehicle Expiration_month': [12, 6, 9, 3, 8],
    'Vehicle Expiration_day': [31, 15, 1, 20, 15],
    'Vehicle Year': [2012, 2010, 2008, 2013, 2014],
    'Hour_sin': [0.5, 0.866, -0.5, 0.0, 0.707],
    'Hour_cos': [0.866, 0.5, 0.866, 1.0, 0.707],
    'Issue Date_year': [2015, 2015, 2015, 2015, 2015],
    'Issue Date_month': [5, 5, 6, 4, 7],
    'Issue Date_day': [15, 20, 1, 10, 4]
})

print("Sample parking violations:")
sample_data[['Registration State', 'Vehicle Make', 'Vehicle Body Type', 'Violation County']].head()

## Make Predictions

In [ ]:
# Preprocess features
X_processed = preprocessor.transform(sample_data)
X_tensor = torch.FloatTensor(X_processed)

# Make predictions
with torch.no_grad():
    # Regression: predict fine amount
    fine_amounts = reg_model(X_tensor).numpy()
    
    # Classification: predict fine category
    class_outputs = class_model(X_tensor)
    _, class_indices = torch.max(class_outputs, 1)
    fine_categories = label_encoder.inverse_transform(class_indices.numpy())

# Add predictions to dataframe
sample_data['Predicted_Fine_Amount'] = fine_amounts.round(2)
sample_data['Predicted_Fine_Category'] = fine_categories

# Display results
results_display = sample_data[[
    'Registration State', 'Vehicle Make', 'Vehicle Body Type',
    'Predicted_Fine_Amount', 'Predicted_Fine_Category'
]]

print("\nPrediction Results:")
print(results_display.to_string(index=False))

## Visualize Predictions

In [ ]:
# Plot predicted fine amounts
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.bar(range(len(sample_data)), sample_data['Predicted_Fine_Amount'])
plt.xlabel('Sample Index')
plt.ylabel('Predicted Fine Amount ($)')
plt.title('Predicted Fine Amounts')
plt.xticks(range(len(sample_data)))

plt.subplot(1, 2, 2)
category_counts = sample_data['Predicted_Fine_Category'].value_counts()
plt.bar(category_counts.index, category_counts.values)
plt.xlabel('Fine Category')
plt.ylabel('Count')
plt.title('Predicted Fine Categories')

plt.tight_layout()
plt.show()

## Model Architecture

Let's examine the neural network architecture.

In [ ]:
print("Regression Model Architecture:")
print("="*50)
print(reg_model)
print(f"\nTotal parameters: {sum(p.numel() for p in reg_model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in reg_model.parameters() if p.requires_grad):,}")

In [ ]:
print("\nClassification Model Architecture:")
print("="*50)
print(class_model)
print(f"\nTotal parameters: {sum(p.numel() for p in class_model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in class_model.parameters() if p.requires_grad):,}")

## Training History

Visualize the training and validation loss over epochs.

In [ ]:
# Plot training history for regression model
if 'history' in reg_checkpoint:
    history = reg_checkpoint['history']
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_losses'], label='Train Loss')
    plt.plot(history['val_losses'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.title('Regression Model Training History')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot for classification model
    if 'history' in class_checkpoint:
        class_history = class_checkpoint['history']
        
        plt.subplot(1, 2, 2)
        plt.plot(class_history['train_losses'], label='Train Loss')
        plt.plot(class_history['val_losses'], label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss (CrossEntropy)')
        plt.title('Classification Model Training History')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Training history not available in checkpoint")

## Summary Statistics

In [ ]:
print("\nPrediction Summary:")
print("="*50)
print(f"Average predicted fine: ${sample_data['Predicted_Fine_Amount'].mean():.2f}")
print(f"Minimum predicted fine: ${sample_data['Predicted_Fine_Amount'].min():.2f}")
print(f"Maximum predicted fine: ${sample_data['Predicted_Fine_Amount'].max():.2f}")
print(f"Std deviation: ${sample_data['Predicted_Fine_Amount'].std():.2f}")

print("\nCategory Distribution:")
print(sample_data['Predicted_Fine_Category'].value_counts())

## Model Performance Metrics

Compare with other models from the main notebook.

In [ ]:
# Create comparison table
comparison = pd.DataFrame({
    'Model': ['PyTorch Neural Network', 'Linear Regression', 'Random Forest', 'XGBoost'],
    'MAE': [reg_checkpoint['metrics']['mae'], 'Run notebook to compare', '', ''],
    'RMSE': [reg_checkpoint['metrics']['rmse'], 'Run notebook to compare', '', ''],
    'R²': [f"{reg_checkpoint['metrics']['r2']:.4f}", 'Run notebook to compare', '', '']
})

print("\nRegression Model Comparison:")
print(comparison.to_string(index=False))

print("\n" + "="*50)
print("\nClassification Model Performance:")
print(f"Accuracy: {class_checkpoint['metrics']['accuracy']:.4f}")
print(f"F1 Score (macro): {class_checkpoint['metrics']['f1_macro']:.4f}")
print("\nTo compare with XGBoost, run the main nyc_fines.ipynb notebook.")

## Conclusion

This notebook demonstrated:
1. Loading trained PyTorch models
2. Making predictions on new data
3. Visualizing results
4. Examining model architecture
5. Viewing training history

### Next Steps:
- Compare these results with models in `nyc_fines.ipynb`
- Experiment with different architectures
- Try hyperparameter tuning
- Use the models for production predictions

### Documentation:
See `pytorch_model_readme.md` for detailed documentation and usage instructions.